In [ ]:
#biblioteca utilizadas para suportar scripts escritos abaixo
import datetime
import re
import locale
import pyodbc 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
import seaborn as sns
from matplotlib.dates import DateFormatter
from matplotlib.ticker import AutoMinorLocator
import sqlalchemy as sa
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
# Para funcionar o pyodbc é necessário adicionar a seguinte linha no registro

cnxn = pyodbc.connect('Driver={SQL Server};'
                      'Server=***;'
                      'Database=***;'
                      'UID=***;'
                      'PWD=***;')
                      

In [ ]:
# Verificar a conexão
cursor = cnxn.cursor()
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

In [ ]:
# Puxando os dados direto do banco do jazz atraves da query a baixo
df = pd.read_sql_query("SELECT DISTINCT T1.REFERENCE_ID, T1.CREATION_DATE, T1.RESOLVED_DATE, T2_1.LITERAL_NAME AS LITERAL_NAME, T1.REQUEST_STATE, T1.REQUEST_SEVERITY, T1.TEAM_NAME, FORMAT(CREATION_DATE,'MMM/yyyy')as CRIACAO_FORMAT, FORMAT(RESOLVED_DATE,'MMM/yyyy')as FINAL_FORMAT FROM RIDW.VW_REQUEST T1 LEFT OUTER JOIN RICALM.VW_RQST_STRING_EXT T2 ON T2.REQUEST_ID=T1.REQUEST_ID AND T2.NAME='Sistema' LEFT OUTER JOIN RICALM.REQUEST_ATTRDEF T2_2 ON T2_2.REQUEST_TYPE_ID=T1.REQUEST_TYPE_ID AND T2_2.NAME=T2.NAME LEFT OUTER JOIN RICALM.VW_RQST_ENUMERATION T2_1 ON T2_1.EXTERNAL_ID=T2.VAL AND T2_1.PROJECT_ID=T1.PROJECT_ID AND T2_1.ENUMERATION_NAME=T2_2.DATATYPE WHERE T1.PROJECT_ID = 415  AND (T1.REQUEST_TYPE = 'Incidente' AND T1.CREATION_DATE >= '2021-07-01' ) AND T1.ISSOFTDELETED = 0 AND (T1.REQUEST_ID <> -1 AND T1.REQUEST_ID IS NOT NULL)", cnxn)
df.to_excel("Dados_RevistaH.xlsx")

In [ ]:
#algoritimo para utilizar no filtro do mes anterior
today=datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
fil=(last_month.strftime("%b/%Y"))

In [ ]:
# Quantidade total de incidentes abertos desde 01/04/2022 // Indicador 0
quant_abertoT=df[df.columns[1]].count()

# Imprimindo o resultado
quant_abertoT

In [ ]:
# substitui os status detalhados por status simples para agregar os dados e posteriormente imprimir a quantidade agregada // importante para o indicador 1
df2 = df.replace({'REQUEST_STATE': {'Concluído': 'Fechado', 'Cancelado': 'Fechado', 'Aguardando Execução': 'Backlog', 'Em Execução': 'Backlog', 'Em Homologação' : 'Backlog', 'E _ Aguardando Times Internos' : 'Backlog', 'Aberto' :'Backlog', 'E _ Aguardando Demandante' : 'Backlog' , 'A _ Aguardando Demandante' : 'Backlog', 'E _ Aguardando Fornecedor':'Backlog'}}).copy()

# Agrupa as linhas da coluna [REFERENCE_ID] por mês utilizando a coluna [CREATION_DATE] demonstrando a quantidade de incidentes fechados e backlog por mes// Indicador 1
opcl=df2
opcl = opcl[opcl['REQUEST_STATE'].isin(['Fechado'])]
fechadoT = opcl.groupby(['REQUEST_STATE']).count()['REFERENCE_ID']

# Imprimindo o resultado
fechadoT



In [ ]:
df_last = df[(df["CRIACAO_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês utilizando a coluna [CREATION_DATE] demonstrando a quantidade de incidentes abertos por mes// Indicador 1
abertoM = df_last.groupby(df['CRIACAO_FORMAT'])['REFERENCE_ID'].count()

# Imprimindo o resultado
abertoM

In [ ]:
# substitui os status detalhados por status simples para agregar os dados e posteriormente imprimir a quantidade agregada // importante para o indicador 1
df2 = df.replace({'REQUEST_STATE': {'Concluído': 'Fechado', 'Cancelado': 'Fechado', 'Aguardando Execução': 'Backlog', 'Em Execução': 'Backlog', 'Em Homologação' : 'Backlog', 'E _ Aguardando Times Internos' : 'Backlog', 'Aberto' :'Backlog', 'E _ Aguardando Demandante' : 'Backlog' , 'A _ Aguardando Demandante' : 'Backlog', 'E _ Aguardando Fornecedor':'Backlog'}}).copy()
df_last = df2[(df2["FINAL_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês utilizando a coluna [CREATION_DATE] demonstrando a quantidade de incidentes fechados e backlog por mes// Indicador 1
opcl=df_last
fechadoM = opcl.groupby([df_last['FINAL_FORMAT'], 'REQUEST_STATE']).count()['REFERENCE_ID']

# Imprimindo o resultado
fechadoM

In [ ]:
opcl=df2
Backlog = opcl[opcl['REQUEST_STATE'].isin(['Backlog'])]

Backlog1 = Backlog.groupby(['REQUEST_STATE']).count()['REFERENCE_ID']

# Imprimindo o resultado
Backlog1

In [ ]:
backlog_st = df
backlog_st0 = backlog_st.replace({'REQUEST_STATE': {'Aguardando Execução': 'Backlog', 'Em Execução': 'Em andamento', 'Em Homologação' : 'Pendentes Validação', 'Em Validação' : 'Pendentes Validação', 'E _ Aguardando Times Internos' : 'Pendentes Outros', 'Aberto' :'Backlog', 'E _ Aguardando Demandante' : 'Pendentes Outros' , 'E _ Aguardando Fornecedor':'Pendentes Outros'}})
backlog_st0 = backlog_st0[(backlog_st0["REQUEST_STATE"] != "Concluído") & (backlog_st0["REQUEST_STATE"] != "Cancelado")]


# Agrupando as linhas selecionadas por status e contando a quantidade de requisições
backlog_st0 = backlog_st0.groupby('REQUEST_STATE')['REFERENCE_ID'].count()

# Imprime o resultado de forma ascendente(do maior para o menor)
print(backlog_st0.sort_values(ascending=False))

In [ ]:
#algoritimo para utilizar no filtro do mes anterior
today=datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
fil=(last_month.strftime("%b/%Y"))

df_last = df[(df["CRIACAO_FORMAT"] == last_month.strftime("%b/%Y"))]

backlog_st1 = df_last
backlog_st1 = backlog_st1.replace({'REQUEST_STATE': {'Aguardando Execução': 'Backlog', 'Em Execução': 'Em andamento', 'Em Homologação' : 'Pendentes Validação', 'Em Validação' : 'Pendentes Validação', 'E _ Aguardando Times Internos' : 'Pendentes Outros', 'Aberto' :'Backlog', 'E _ Aguardando Demandante' : 'Pendentes Outros' , 'E _ Aguardando Fornecedor':'Pendentes Outros'}})
#backlog_st = backlog_st[(backlog_st["REQUEST_STATE"] != "Concluído") & (backlog_st["REQUEST_STATE"] != "Cancelado")]
# Agrupando as linhas selecionadas por status e contando a quantidade de requisições
backlog_st1 = backlog_st1.groupby([df_last['CRIACAO_FORMAT'], 'REQUEST_STATE']).count()['REFERENCE_ID']
# Imprime o resultado de forma ascendente(do maior para o menor)
backlog_st1

In [ ]:
#algoritimo para utilizar no filtro do mes anterior
today=datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
fil=(last_month.strftime("%b/%Y"))

df_last = df[(df["CRIACAO_FORMAT"] == last_month.strftime("%b/%Y"))]

backlog_st2 = df_last
backlog_st2 = backlog_st2.replace({'REQUEST_STATE': {'Concluído': 'Fechado', 'Cancelado': 'Fechado','Aguardando Execução': 'Backlog', 'Em Execução': 'Em andamento', 'Em Homologação' : 'Pendentes Validação', 'Em Validação' : 'Pendentes Validação', 'E _ Aguardando Times Internos' : 'Pendentes Outros', 'Aberto' :'Backlog', 'E _ Aguardando Demandante' : 'Pendentes Outros' , 'E _ Aguardando Fornecedor':'Pendentes Outros'}})
backlog_st2 = backlog_st2[backlog_st2['REQUEST_STATE'].isin(['Fechado'])]
# Agrupando as linhas selecionadas por status e contando a quantidade de requisições
FechadoCM = backlog_st2.groupby([df_last['CRIACAO_FORMAT'], 'REQUEST_STATE']).count()['REFERENCE_ID']
# Imprime o resultado de forma ascendente(do maior para o menor)
FechadoCM

In [ ]:
Taxa_atendimento= FechadoCM/abertoM
Taxa_atendimento

In [ ]:
filename = 'Historico_incidentes.xlsx'
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# In[3]:
wb = Workbook()
ws = wb.active
# Cria DataFrame para o valor de quant_abertoT
quant_abertoT_df = pd.DataFrame({'Total de incidentes abertos': [quant_abertoT]})
# Salva o DataFrame em uma planilha no arquivo Excel
quant_abertoT_df.to_excel(writer, sheet_name='Total de incidentes abertos')
fechadoT.to_excel(writer, sheet_name='Total de incidentes fechados')
abertoM.to_excel(writer, sheet_name='abertos no mes')
fechadoM.to_excel(writer, sheet_name='fechados no mes')
Backlog1.to_excel(writer, sheet_name='backlog no mes')
backlog_st0.to_excel(writer, sheet_name='backlog st agrp')
backlog_st1.to_excel(writer, sheet_name='backlog st detal')
FechadoCM.to_excel(writer, sheet_name='fechados do mes')
Taxa_atendimento.to_excel(writer, sheet_name='taxa de atnedimento')

writer.save()
print(f"Os dados foram salvos em {filename} com sucesso!")